# Team Allocation Simulator #
by Ewen, Wei Hao, Zerius, Youde, Shawn

## Resources provided ##
We have received a CSV file 'records.csv', which have given us the following information of each of the 6000 students
1. The students' name
2. The students' ID
3. The students' tutorial group
4. The students' gender
5. The students' school affiliation
6. The students' current CGPA

## Requirements ##
We are to develop an application that does the following 
1. Organize students into teams of five
2. Team members should be in the same tutorial group
3. No team should have a majority of students from the same school
4. No team should have a majority of students of the same gender
5. No team should not consist predominantly of students with very high or very low CGPAs 

## Thought Process ##
We have analysed the current context and decided that we should follow this procedure
1. Develop a function to read CSV files and extract students's information.
2. Develop a function to sort students by tutorial groups
3. Develop a function to add students into a team, while ensuring there is a balance in gender and there are no students with the same school affliation, as well as have a mixture of CGPAs.
4. Develop a function to sort students by gender, then CGPA in their own respective tutorial groups

We believe that this would be the most efficient approach

## Function to Read CSV File ##
We will first create a function to read the CSV. It will then add each student's information (those listed above) into a dictionary. The dictionaries of the students' details will be stored in a list.

In [1]:
def read_student_data(file_path):
    """Read data from csv file and return"""
    students_list = []
    with open(file_path, mode='r') as file:
        headers = file.readline().strip().split(",")
        for line in file:
            student = {}
            values = line.strip().split(",")
            for i, header in enumerate(headers):
                student[header.strip()] = values[i].strip()
                if header.strip() == "CGPA":
                    student[header.strip()] = float(student[header.strip()])

            students_list.append(student)

    return students_list

## Function to Sort Students By Tutorial Groups ##
Since the members of each group should be in the same tutorial group, we decided that it would be more efficient to meet this requirement first. In this function, we have split the dictionaries of each student to lists of their own respective tutorial groups.

In [2]:
def pick_student(all_students, tut_grp):
    """pick student out from the tutorial group"""
    students_list = []

    for student in all_students:
        if student['Tutorial Group'] == f'G-{tut_grp}':
            students_list.append(student)

    return students_list

## Function to form a team of 5 students ##
We will now design a function to form a team of 5 students. This function will have 2 parameters, a list with male students with their CGPA sorted, as well as a list with female students with their CGPA sorted. We will implement another function later for the sorting of gender and CGPA.

For this function, we will be primarily be focussing on how we would ensure teams would have students with different school affliations, and how we would balance out the CGPA and gender in each group.

Our method would be to select a student with the lowest CGPA amongst his/her gender. We will then select a student of opposite gender with the highest CGPA. This repeats for another time. The final student selected would have the median CGPA amongst his/her gender, where the number of students of this gender is greater than the other. This effectively allows for a good mix of CGPA in a group, while ensuring there will be a 3:2 ratio for the gender and that students have different school affiliations

In the event where the student selected has the same school affliation as another student already in the group, the function will choose the next best alternative
> If the next female student with the lowest CGPA has the same school affiliation with another student in the group, the second lowest CGPA female student will be selected instead. If the second lowest CGPA female student also has same school affilation with another student in the group, the third lowest CGPA female student will be selected, and so on.

However, to ensure that the mean of CGPA does not differ alot between teams, we would stop the changing of index after it increases by 5. This also ensures that the mixture of CGPA will not affect other teams as well.



### Pseudocode for Function ###
<font color='green'>FUNCTION</font> diverse_team(sorted_student_more, sorted_student_less)<br>
&emsp;<font color='green'>INITIALISE</font> team <font color='green'>AS</font> an empty list<br>
&emsp;<font color='green'>SET</font> i <font color='green'>TO</font> 0<br>
&emsp;<font color='green'>INITIALISE</font> sorted_student <font color='green'>AS</font> a list containing sorted_student_more and sorted_student_less<br>

&emsp;<font color='green'>SET</font> gender_run_two_time <font color='green'>TO</font> FALSE<br>
&emsp;<font color='green'>SET</font> next_gender_same <font color='green'>TO</font> FALSE<br>
&emsp;<font color='green'>IF</font> (length of sorted_student_more - length of sorted_student_less >= 10-team_index) <font color='green'>THEN</font><br>
&emsp;&emsp;<font color='green'>SET</font> gender_run_two_time <font color='green'>TO</font> TRUE

&emsp;<font color='green'>SET</font> gender_index <font color='green'>TO</font> 0<br>

&emsp;<font color='green'>WHILE</font> (length of team < 5) <font color='blue'>AND</font> (sorted_student[0] <font color='blue'>OR</font> sorted_student[1])<br>
&emsp;&emsp;<font color='green'>WHILE</font> index_changes <font color='green'>TO</font> [+1, -1, +1]<br>
&emsp;&emsp;<font color='green'>SET</font> pick_index <font color='green'>TO</font> [0, -1, length of sorted_student[gender_index] // 2]<br>

&emsp;&emsp;<font color='green'>WHILE</font> TRUE<br>
&emsp;&emsp;&emsp;<font color='green'>IF</font> sorted_student[gender_index] is empty <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>BREAK</font><br>

&emsp;&emsp;&emsp;<font color='green'>IF</font> (index_changes[i] >= 0) <font color='blue'>AND</font> (pick_index[i] + index_changes[i] < length of sorted_student[gender_index] <font color='blue'>AND</font> index_changes[i] <= 5 <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>IF</font> sorted_student[gender_index][pick_index[i] + index_changes[i]]['School'] <font color='blue'>NOT</font> <font color='green'>IN</font> [student['School'] <font color='green'>FOR</font> student <font color='green'>IN</font> team] <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>APPEND</font> sorted_student[gender_index].<font color='green'>POP</font>(pick_index[i] + index_changes[i]) <font color='green'>TO</font> team<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>BREAK</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>ELSE</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>INCREMENT</font> index_changes[i] <font color='green'>BY</font> 1<br>
&emsp;&emsp;&emsp;<font color='green'>ELSE IF</font> (index_changes[i] < 0) <font color='blue'>AND</font> (ABS(pick_index[i] + index_changes[i]) <= length of sorted_student[gender_index]) <font color='blue'>AND</font> (ABS(index_changes[i]) <= 5) <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>IF</font> (sorted_student[gender_index][pick_index[i] + index_changes[i]]['School']) <font color='blue'>NOT</font> <font color='green'>IN</font> [student['School'] <font color='green'>FOR</font> student <font color='green'>IN</font> team] <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>APPEND</font> sorted_student[gender_index].<font color='green'>POP</font>(pick_index[i] + index_changes[i]) <font color='green'>TO</font> team<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>BREAK</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>ELSE</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>DECREMENT</font> index_changes[i] <font color='green'>BY</font> 1<br>
&emsp;&emsp;&emsp;<font color='green'>ELSE</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;<font color='green'>APPEND</font> sorted_student[gender_index].<font color='green'>POP</font>(pick_index[i]) <font color='green'>TO</font> team<br>
&emsp;&emsp;&emsp;&emsp;&emsp;BREAK<br>

&emsp;&emsp;<font color='green'>INCREMENT</font> i <font color='green'>BY</font> 1

&emsp;&emsp;<font color='green'>IF</font> <font color='blue'>NOT</font> next_gender_same <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;<font color='green'>INCREMENT</font> gender_index <font color='green'>BY</font> 1

&emsp;&emsp;<font color='green'>ELSE IF</font> next_gender_same <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;<font color='green'>SET</font> next_gender_same <font color='green'>TO</font> FALSE

&emsp;&emsp;<font color='green'>IF</font> i is greater than 2 <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;<font color='green'>SET</font> i <font color='green'>TO</font> 0

&emsp;&emsp;<font color='green'>IF</font> gender_index is greater than 1 <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;<font color='green'>SET</font> gender_index <font color='green'>TO</font> 0

&emsp;&emsp;<font color='green'>IF</font> gender_run_two_time <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;<font color='green'>SET</font> gender_index <font color='green'>TO</font> 0<br>
&emsp;&emsp;&emsp;<font color='green'>SET</font> gender_run_two_time <font color='green'>TO</font> TRUE<br>
&emsp;&emsp;&emsp;<font color='green'>SET</font> next_gender_same <font color='green'>TO</font> TRUE

&emsp;<font color='green'>RETURN</font> team<br>

In [ ]:
def diverse_team(sorted_student_more, sorted_student_less, team_index):  # index(0,-1,or middle)
    team = []
    i = 0  # round
    sorted_student = [sorted_student_more, sorted_student_less]

    gender_run_two_time = False
    next_gender_same = False
    if len(sorted_student_more) - len(sorted_student_less) >= 10-team_index:
        gender_run_two_time = True

    gender_index = 0

    while len(team) < 5 and (sorted_student[0] or sorted_student[1]):  # add student into team when teams is not full
        index_changes = [+1, -1, +1]
        pick_index = [0, -1, len(sorted_student[gender_index]) // 2]
        while True:
            if not sorted_student[gender_index]:
                break
            elif index_changes[i] >= 0 and pick_index[i] + index_changes[i] < len(sorted_student[gender_index]) and index_changes[
                i] <= 5:
                # Check if school is not in the team
                if sorted_student[gender_index][pick_index[i] + index_changes[i]]['School'] not in [student['School'] for student in
                                                                                                    team]:
                    team.append(sorted_student[gender_index].pop(pick_index[i] + index_changes[i]))
                    break
                else:
                    index_changes[i] += 1
            elif index_changes[i] < 0 and abs(pick_index[i] + index_changes[i]) <= len(sorted_student[gender_index]) and abs(
                    index_changes[i]) <= 5:
                # Check if school is not in the team
                if sorted_student[gender_index][pick_index[i] + index_changes[i]]['School'] not in [student['School'] for student in
                                                                                                    team]:
                    team.append(sorted_student[gender_index].pop(pick_index[i] + index_changes[i]))
                    break
                else:
                    index_changes[i] -= 1
            else:
                # If the index exceeds the list length, reset index_changes or break
                team.append(sorted_student[gender_index].pop(pick_index[i]))
                break

        i += 1
        if not next_gender_same:
            gender_index += 1
        elif next_gender_same:
            next_gender_same = False
        if i > 2:
            i = 0
        if gender_index > 1:
            gender_index = 0

        if gender_run_two_time:
            gender_index = 0
            gender_run_two_time = False
            next_gender_same = True

    return team


## Function to Sort by Gender and CGPA ##
This function would split students by their gender into 2 lists. Each list will then be sorted by CGPA. The function will determine whether the tutorial group has more males or femeles, and the lists produced will then be used as diverse_team parameters. 
The function will return the groups formed in lists, nested in a whole list.

<font color='green'>FUNCTION</font> division_into_team(students_list)<br>
&emsp;<font color='green'>INITIALISE</font> male_students <font color='green'>AS</font> a list of all male students<br>
&emsp;<font color='green'>INITIALISE</font> female_students <font color='green'>AS</font> a list of all female students

&emsp;<font color='green'>INITIALISE</font> sorted_male_students <font color='green'>AS</font> a list of all male students sorted by their CGPA<br>
&emsp;<font color='green'>INITIALISE</font> sorted_female_students <font color='green'>AS</font> a list of all female students sorted by their CGPA<br>

&emsp;<font color='green'>INITIALISE</font> teams <font color='green'>AS</font> a list of X empty teams, where X is the total number of teams formed in a tutorial group<br>
&emsp;<font color='green'>SET</font> team_index <font color='green'>TO</font> 0<br>

&emsp;<font color='green'>IF</font> (team_index < length of teams) <font color='green'>THEN</font><br>
&emsp;&emsp;<font color='green'>IF</font> (length of male_sorted_students >= length of female_sorted_students) <font color='green'>THEN</font><br>
&emsp;&emsp;&emsp;&emsp;<font color='green'>ASSIGN</font> teams[team_index] <font color='green'>AS</font> diverse_team(male_sorted_students, female_sorted_students)<br>
&emsp;&emsp;<font color='green'>ELSE</font><br>
&emsp;&emsp;&emsp;&emsp;<font color='green'>ASSIGN</font> teams[team_index] <font color='green'>AS</font> diverse_team(female_sorted_students, male_sorted_students)<br>

&emsp;&emsp;<font color='green'>INCREMENT</font> team_index BY 1<br>

&emsp;<font color='green'>RETURN</font> teams<br>


In [4]:
def division_into_team(students_list):
    """Divides student from student_list into group of 5 with balance of cgpa and gender, and diverse of school"""
    # Separate male and female students into two lists
    male_students = [student for student in students_list if student['Gender'] == 'Male']
    female_students = [student for student in students_list if student['Gender'] == 'Female']

    # Sort the male and female students by CGPA
    male_sorted_students = sorted(male_students, key=lambda student: student['CGPA'])
    female_sorted_students = sorted(female_students, key=lambda student: student['CGPA'])

    teams = [[] for _ in range(len(students_list) // 5)]
    team_index = 0

    while team_index < len(teams):  # while haven't added student into all teams
        if len(male_sorted_students) >= len(female_sorted_students):  # if remain male student more than remain female student
            teams[team_index] = diverse_team(male_sorted_students, female_sorted_students)

        else:  # if remain male student less than remain female student
            teams[team_index] = diverse_team(female_sorted_students, male_sorted_students)

        team_index += 1

    return teams

In [5]:
#This function can ignore
def see_group_division(group_div):
    for i, x in enumerate(group_div):
        print(f"Group {i}")
        cgpa = 0
        for j in x:
            print(j["Student ID"], j["School"], j["CGPA"], j["Gender"])
            cgpa += j["CGPA"]
        print(f" the mean cgpa of this group is {cgpa / len(x)}")
        print()


students = read_student_data('records.csv')
group_division = []
tutorial_grp = 1

while tutorial_grp <= 150:
    group_division.extend(division_into_team(pick_student(students, tutorial_grp)))
    tutorial_grp += 1

see_group_division(group_division)

Group 0
2353 SBS 3.95 Female
2091 EEE 4.2 Male
592 MSE 4.11 Female
1645 CoE 3.93 Male
2069 SSS 4.48 Female
 the mean cgpa of this group is 4.134

Group 1
2326 CoB (NBS) 3.95 Female
3989 WKW SCI 4.15 Male
4479 CCDS 4.11 Female
3148 EEE 3.88 Male
4338 SPMS 4.22 Female
 the mean cgpa of this group is 4.061999999999999

Group 2
4563 WKW SCI 4.01 Female
2776 CCEB 4.14 Male
5703 SPMS 4.12 Female
4657 SoH 4.0 Male
809 CoB (NBS) 4.26 Female
 the mean cgpa of this group is 4.106

Group 3
2151 EEE 4.02 Female
1841 MAE 4.12 Male
1271 SSS 4.17 Female
288 CoB (NBS) 4.01 Male
4820 CoE 4.22 Female
 the mean cgpa of this group is 4.108

Group 4
2115 EEE 4.03 Female
1417 CoE 4.12 Male
945 MAE 4.1 Female
5002 CCDS 4.02 Male
659 SSS 4.2 Female
 the mean cgpa of this group is 4.093999999999999

Group 5
567 CoB (NBS) 4.03 Female
4520 EEE 4.11 Male
2650 SoH 4.09 Female
588 MAE 4.06 Male
4402 CCDS 4.08 Female
 the mean cgpa of this group is 4.074

Group 6
75 CCDS 4.03 Female
5477 SoH 4.09 Male
3930 EEE 4.18 